In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib as mlp
from tkinter import Tcl
import glob
import json
from collections import Counter

In [2]:
##is the problem scaled??

scaled=True

In [3]:
#defining image saving function

def save_fig(_id, tight_layout=True, resolution=300):
    path = os.path.join("./output/Adaptibility", _id + ".png")
    print("Saving figure", _id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=resolution)

In [4]:
#loading user data and getting number of stages

user_data=pd.read_csv('input/user_data.csv')
n_stages=user_data.iloc[0,1]
user_date_start=str(2020)+'-'+str(user_data.iloc[1,1])+'-'+str(user_data.iloc[2,1])
user_date_start = pd.to_datetime(user_date_start)
user_date_end=user_date_start+pd.DateOffset(user_data.iloc[0,1])
print(f'Start date:    {user_date_start}')
user_data

Start date:    2020-03-31 00:00:00


variables  values
0                                   Number of stages      21
1                          month for initial storage       3
2                            day for initial storage      31
3  nuber of rows from begening of USACE forecsat ...       3
4   nuber of rows from End of USACE forecsat to drop       0
5                              number of small trees       7

In [5]:
params=pd.read_excel('input/Cascade_info.xlsx',sheet_name='plotting_info')
params=params.sort_values(by='name',ascending=True)
params=params.reset_index(drop=True)
name=params['name']
name=sorted(name)
name

['BB', 'FP', 'FR', 'GA', 'GP', 'OH']

In [6]:
params

name      k  max_powerhouse  max_spill  baseline  flood_control  \
0   BB   4.60          103000     270000   1810000        1749000   
1   FP   5.84           31721     230000  18463000       17492000   
2   FR   8.50           37647     508000   5293000        4307000   
3   GA  12.30           41000     660000  23451000       21956000   
4   GP   3.45           36000     345000    428000         374000   
5   OH  12.45           54600      80000  22983000       21876000   

   release_min  release_max  
0            0       103000  
1            6        39200  
2            9        44496  
3            9        41000  
4            9        36000  
5            0        54000

In [7]:
storage_max=params['flood_control']
re_max=params['release_max']

In [8]:
storage_max

0     1749000
1    17492000
2     4307000
3    21956000
4      374000
5    21876000
Name: flood_control, dtype: int64

In [9]:
#making list of actual release and storage for time period
df_list=[]
for n in name:
    temp=pd.read_excel("input/Ground_truth.xlsx",sheet_name=n)
    c=list(temp.columns)
    c.remove('Date')
    c.remove('Storage')
    c.remove('Release')
    temp.drop(columns=c,inplace=True)
    row=temp.index[temp['Date']==user_date_start]
    temp=temp[row[0]+1:row[0]+1+n_stages]
    df_list.append(temp)

In [10]:
df_list[0]

Date  Release  Storage
91  2020-04-01    36300  1672000
92  2020-04-02    35900  1662000
93  2020-04-03    32200  1649000
94  2020-04-04    20400  1656000
95  2020-04-05    15200  1665000
96  2020-04-06    28500  1660000
97  2020-04-07    28400  1659000
98  2020-04-08    29500  1658000
99  2020-04-09    29600  1660000
100 2020-04-10    29800  1658000
101 2020-04-11    18000  1677000
102 2020-04-12    19500  1688000
103 2020-04-13    29500  1681000
104 2020-04-14    29800  1675000
105 2020-04-15    29800  1669000
106 2020-04-16    30300  1664000
107 2020-04-17    30400  1657000
108 2020-04-18    17400  1674000
109 2020-04-19    16600  1686000
110 2020-04-20    32500  1675000
111 2020-04-21    33400  1670000

In [11]:
cur_dir=os.getcwd()
cur_dir
so_tree_path=os.path.join(cur_dir,'output/solution_scenarios')
so_tree_path

'C:\\Users\\chrs-164-ip\\Desktop\\Projects\\SDDP\\General structure\\For github\\output/solution_scenarios'

In [12]:
so_tree_dict={}
for dam in name:
    so_tree_dict[dam]=""

In [13]:
so_tree_dict

{'BB': '', 'FP': '', 'FR': '', 'GA': '', 'GP': '', 'OH': ''}

In [14]:
file_end=('release.csv','storage.csv')
for n in name:
    temp_list=[]
    for f in os.listdir(so_tree_path):
        f_path=os.path.join(so_tree_path,f)
        if f.startswith(n) and f.endswith(file_end):
            temp=pd.read_csv(f_path,header=None)
            temp_list.append(temp)
        so_tree_dict[n]=temp_list

In [15]:
os.listdir(so_tree_path)

['BB_solution_scenarios_release.csv',
 'BB_solution_scenarios_spill.csv',
 'BB_solution_scenarios_storage.csv',
 'BB_solution_scenarios_total_pflow.csv',
 'FP_solution_scenarios_release.csv',
 'FP_solution_scenarios_spill.csv',
 'FP_solution_scenarios_storage.csv',
 'FP_solution_scenarios_total_pflow.csv',
 'FR_solution_scenarios_release.csv',
 'FR_solution_scenarios_spill.csv',
 'FR_solution_scenarios_storage.csv',
 'FR_solution_scenarios_total_pflow.csv',
 'GA_solution_scenarios_release.csv',
 'GA_solution_scenarios_spill.csv',
 'GA_solution_scenarios_storage.csv',
 'GA_solution_scenarios_total_pflow.csv',
 'GP_solution_scenarios_release.csv',
 'GP_solution_scenarios_spill.csv',
 'GP_solution_scenarios_storage.csv',
 'GP_solution_scenarios_total_pflow.csv',
 'OH_solution_scenarios_release.csv',
 'OH_solution_scenarios_spill.csv',
 'OH_solution_scenarios_storage.csv',
 'OH_solution_scenarios_total_pflow.csv']

In [16]:
##lets check on of them

so_tree_dict[name[0]][0]

0          1          2          3          4          5   \
0   78.941147  78.941147  78.941147  78.941147  78.941147  78.941147   
1   53.322139  53.322139  53.322139  50.205275  53.322139  50.205275   
2   47.028735  47.028735  47.028735  43.053229  47.028735  43.053229   
3   55.200000  55.200000  55.200000  54.600000  55.200000  54.600000   
4   54.600000  54.600000  54.600000  54.600000  54.600000  54.600000   
5   54.600000  54.600000  54.600000  54.600000  54.600000  54.600000   
6   54.680000  54.680000  54.680000  54.600000  54.680000  58.240000   
7   54.600000  54.600000  54.600000  52.237831  54.600000  38.667443   
8   54.700000  54.700000  54.700000  15.458924  54.700000  30.407118   
9   26.834586  26.834586  26.834586  85.802763  26.834586  24.062485   
10  54.969760  54.969760  54.969760  50.038984  54.969760  26.238639   
11  37.731394  37.731394  37.731394  53.994709  37.731394  23.165189   
12  34.168782  34.168782  34.168782  49.673064  34.168782  18.595893   
13  54.975378  54.975378  54.975378  57.000000  54.975378  40.692043   
14  54.984622  17.713597  54.984622  55.840000  54.984622  47.665352   
15  33.667699  56.087906  33.667699  29.715269  33.667699  24.660646   
16  54.600000  54.600000  54.600000  54.600000  54.600000  42.335755   
17  54.600000  54.600000  17.597495  54.600000  54.600000  57.805565   
18  37.021720  28.754213  21.594629  49.551552  26.301611  21.648475   
19  29.155918  27.814294  41.621596  51.623166  37.265929  10.645729   
20  32.424273  25.909340  16.062158  48.888980  33.941981   1.543210   

           6          7          8          9   ...          20         21  \
0   78.941147  78.941147  78.941147  78.941147  ...   78.941147  78.941147   
1   50.205275  53.322139  53.322139  50.205275  ...   50.205275  50.205275   
2   43.053229  47.028735  47.028735  43.053229  ...   43.053229  37.897183   
3   54.600000  55.200000  55.200000  54.600000  ...   54.600000  65.541418   
4   54.600000  54.600000  54.600000  54.600000  ...   54.600000  58.961986   
5   54.600000  54.600000  57.200000  54.600000  ...   54.600000  55.999414   
6   54.600000  54.680000  56.760000  54.600000  ...   54.600000  55.222000   
7   52.237831  54.600000  43.141706  52.237831  ...   52.237831  54.600000   
8   15.458924  55.600000  10.075277  15.458924  ...   15.458924   9.895992   
9   85.802763  41.694667  73.759743  85.802763  ...   85.802763  90.730996   
10  50.038984  31.390005  41.084549  50.038984  ...   50.038984  51.338894   
11  53.994709  32.352808  41.727767  53.994709  ...   53.994709  46.297751   
12  49.673064  45.298959  51.155020  49.673064  ...   49.673064  48.965291   
13  57.000000  56.742316  54.600000  57.000000  ...   57.000000  51.619785   
14  55.840000  54.600000  54.880000  55.840000  ...   55.840000  57.580215   
15  29.715269  36.696069  31.926901  29.715269  ...    1.543210  32.630403   
16  56.660000  62.280000  54.940000  54.600000  ...  101.646091  54.600000   
17  58.560000  50.823045  54.600000  31.789486  ...   60.630700  54.600000   
18  55.230716  57.839269  42.116614  39.677435  ...   54.600000  49.825736   
19  53.819578  40.830513  29.913795  16.642315  ...   18.892794  46.074417   
20  13.902811  26.634287  25.719895   1.543210  ...   20.878078  35.169823   

           22         23         24         25         26         27  \
0   78.941147  78.941147  78.941147  78.941147  78.941147  78.941147   
1   50.205275  53.322139  53.322139  50.205275  53.322139  53.322139   
2   37.897183  47.028735  47.028735  37.897183  47.028735  47.028735   
3   71.342817  55.200000  55.200000  71.342817  55.200000  55.200000   
4   54.620000  54.600000  54.600000  54.620000  54.600000  54.600000   
5   45.477586  54.600000  54.600000  45.477586  54.600000  54.600000   
6   54.660000  54.680000  54.680000  54.660000  54.680000  54.680000   
7   54.600000  54.600000  54.600000  54.600000  54.600000  54.600000   
8   55.120000   6.484055  54.700000  55.120000  54.700000  54

In [17]:
##scaling the dfs if needed

if scaled:
    print('is scaled!')
    so_tree_dict['BB'][0]
    for n in name:
        so_tree_dict[n][0]=so_tree_dict[n][0]*1e3
        so_tree_dict[n][1]=so_tree_dict[n][1]*1e6

is scaled!


In [18]:
##defining plot function

def plot_d(arr,xlabel=name,storage_max=storage_max,re_max=re_max):
    release=[]
    storage=[]
    
    ##defining storage_max and release_max
    
    ##making the lists
    for i in range(arr.shape[1]):
        temp_r=arr[0,i,:]
        temp_s=arr[1,i,:]
        release.append(temp_r)
        storage.append(temp_s)
    
    ## box plot of release
    fig=plt.figure(figsize=(40,20),dpi=300)
    ax=fig.add_subplot(111,label='1')
    plt.boxplot(release, patch_artist=True,
            boxprops=dict(facecolor='b', color='b',alpha=.5,linewidth=5),
            medianprops=dict(color='k',linewidth=5),
            )
    plt.scatter(ax.get_xticks(),re_max,marker='*',s=1000,color='r',label='Maximum Release')
    plt.legend(fontsize=40)
    plt.grid(True,which='major',axis='both')
    ax.set_xticklabels(xlabel,fontsize=30)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.ylabel('CFs',fontsize=40)
    save_fig('Release adaptibility')
    
    ## box plot of storage
    fig2=plt.figure(figsize=(40,20),dpi=300)
    ax=fig2.add_subplot(111,label='1')
    plt.boxplot(storage, patch_artist=True,
            boxprops=dict(facecolor='b', color='b',alpha=.5,linewidth=5),
            medianprops=dict(color='k',linewidth=5),
            )
    plt.grid(True,which='major',axis='both')
    ax.set_xticklabels(xlabel,fontsize=40)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.ylabel('AF',fontsize=40)
    save_fig('Storage adaptibility1')
    
    
    ## box plot of storage with max storage
    fig3=plt.figure(figsize=(40,20),dpi=300)
    ax=fig3.add_subplot(111,label='1')
    storage1=[]
    storage1=[qq /1e6 for qq in storage]
    storage1m=[]
    storage1m=[qqq /1e6 for qqq in storage_max]
    plt.boxplot(storage1, patch_artist=True,
            boxprops=dict(facecolor='b', color='b',alpha=.5,linewidth=5),
            medianprops=dict(color='k',linewidth=5),
            )
    ax.bar(ax.get_xticks(),storage1m,color='r',alpha=.3, align='center')
    
    plt.grid(True,which='major',axis='both')
    ax.set_xticklabels(xlabel,fontsize=40)
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.ylabel('MAF',fontsize=40)
    save_fig('Storage adaptibility2')
    return release,storage

In [19]:
##making empty array for release and storage

d_history=np.zeros([2,len(name),so_tree_dict[name[0]][0].shape[1]])*np.nan
d_history2=np.zeros([2,len(name),so_tree_dict[name[0]][0].shape[1]])*np.nan

for n,i in zip(name,range(len(df_list))):
    ##geting actual values for each dam
    act_release=df_list[i].iloc[:,1].to_numpy()
    act_storage=df_list[i].iloc[:,2].to_numpy()
    ##geting sim values for each dam
    sim_release_df=so_tree_dict[n][0]
    sim_storage_df=so_tree_dict[n][1]
    for sce in range(len(sim_release_df.columns)):
        d_r2=np.sum(abs(sim_release_df.iloc[:,sce]-act_release))/len(act_release)
        d_s2=np.sum(abs(sim_storage_df.iloc[:,sce]-act_storage))/len(act_storage)
        
        ##for release
        d_history2[0,i,sce]=d_r2 
        ##for storage
        d_history2[1,i,sce]=d_s2 #storage

##ploting the results and geting the output arrays to save
r,s=plot_d(d_history2)        
    

Saving figure Release adaptibility
Saving figure Storage adaptibility1
Saving figure Storage adaptibility2


In [20]:
##converting list to array

r_arr=np.array(r)
s_arr=np.array(s)

In [21]:
##saving the array

np.save("output/temp_files/release_boxplot_arr.npy",r_arr)
np.save("output/temp_files/storage_boxplot_arr.npy",s_arr)

In [22]:
plt.close('all')

In [23]:
print('code is Done!')

code is Done!
